In [3]:
%pip install dash --user
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output

# Datensatz laden
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/uber-rides-data1.csv')

# Dash-App erstellen (für interaktive Visualisierung)
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Uber Rides: Interaktive Visualisierung"),
    dcc.Graph(id='overview_map'),
    html.P("Wählen Sie eine Region aus, um zu zoomen und zu filtern:"),
    dcc.RangeSlider(
        id='lon_range', min=df['Lon'].min(), max=df['Lon'].max(),
        step=0.01, value=[df['Lon'].min(), df['Lon'].max()],
        marks={i: str(i) for i in range(int(df['Lon'].min()), int(df['Lon'].max())+1)},
    ),
    dcc.RangeSlider(
        id='lat_range', min=df['Lat'].min(), max=df['Lat'].max(),
        step=0.01, value=[df['Lat'].min(), df['Lat'].max()],
        marks={i: str(i) for i in range(int(df['Lat'].min()), int(df['Lat'].max())+1)},
    ),
    dcc.Graph(id='zoomed_map'),
    html.H3("Details auf Anfrage"),
    dcc.Graph(id='details')
])

@app.callback(
    Output('overview_map', 'figure'),
    Input('lon_range', 'value'),
    Input('lat_range', 'value')
)
def update_overview(lon_range, lat_range):
    filtered_df = df[(df['Lon'] >= lon_range[0]) & (df['Lon'] <= lon_range[1]) &
                     (df['Lat'] >= lat_range[0]) & (df['Lat'] <= lat_range[1])]
    fig = px.scatter_mapbox(filtered_df, lat='Lat', lon='Lon', zoom=10)
    fig.update_layout(mapbox_style="open-street-map")
    return fig

@app.callback(
    Output('zoomed_map', 'figure'),
    Input('overview_map', 'selectedData')
)
def update_zoomed(selectedData):
    if selectedData is None:
        return px.scatter_mapbox(df, lat='Lat', lon='Lon', zoom=10)

    lons = [point['lon'] for point in selectedData['points']]
    lats = [point['lat'] for point in selectedData['points']]
    zoomed_df = df[df['Lon'].isin(lons) & df['Lat'].isin(lats)]
    fig = px.scatter_mapbox(zoomed_df, lat='Lat', lon='Lon', zoom=13)
    fig.update_layout(mapbox_style="open-street-map")
    return fig

@app.callback(
    Output('details', 'figure'),
    Input('zoomed_map', 'clickData')
)
def display_click_data(clickData):
    if clickData is None:
        return go.Figure()

    clicked_point = df[(df['Lon'] == clickData['points'][0]['lon']) & 
                       (df['Lat'] == clickData['points'][0]['lat'])]
    fig = go.Figure(data=[go.Bar(x=clicked_point.columns, y=clicked_point.iloc[0])])
    fig.update_layout(title='Detailinformationen des ausgewählten Punktes')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached dash-2.14.2-py3-none-any.whl.metadata (11 kB)
  Using cached flask-3.0.0-py3-none-any.whl.metadata (3.6 kB)
Using cached dash-2.14.2-py3-none-any.whl (10.2 MB)
Using cached flask-3.0.0-py3-none-any.whl (99 kB)
